# Задача

Реализовать класс `MyBinaryLogisticRegression` для работы с логистической регрессией. Обеспечить возможность использования `l1`, `l2` и `l1l2` регуляризации и реализовать слудующие методы решения оптимизационной задачи:

*   Градиентный спуск
*   Стохастический градиентный спуск
*   Метод Ньютона

Обосновать применимость/не применимость того или иного метода оптимизации в случае использованного типа регуляризации.



In [48]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings

In [55]:
class MyBinaryLogisticRegression:
    def __init__(self, l1_ratio=0.0, alpha=0.01, solver='gd',
                 iters=1000, lr=0.1, tol=1e-4):
        """
        l1_ratio: 0 - L2, 1 - L1, (0, 1) - Elastic Net
        alpha: сила регуляризации
        solver: 'gd', 'sgd', 'newton'
        """
        self.l1_ratio = l1_ratio
        self.alpha = alpha
        self.solver = solver
        self.iters = iters
        self.lr = lr
        self.tol = tol
        self.coefs_ = None
        self.intercept_ = None
        self.feature_names_in_ = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -250, 250)))

    def _get_reg_term(self, w):
        w_for_reg = np.copy(w)
        w_for_reg[0] = 0

        l1_grad = self.l1_ratio * self.alpha * np.sign(w_for_reg)
        l2_grad = (1 - self.l1_ratio) * self.alpha * w_for_reg
        return l1_grad + l2_grad

    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.feature_names_in_ = X.columns.tolist()
        X_matrix = X.values
        y_matrix = y.values.reshape(-1, 1)

        n_samples, n_features = X_matrix.shape
        weights = np.zeros((n_features + 1, 1))
        X_inc = np.hstack([np.ones((n_samples, 1)), X_matrix])

        if self.solver == 'gd':
            weights = self._gradient_descent(X_inc, y_matrix, weights)
        elif self.solver == 'sgd':
            weights = self._stochastic_gradient_descent(X_inc, y_matrix, weights)
        elif self.solver == 'newton':
            weights = self._newton_method(X_inc, y_matrix, weights)
        else:
            raise ValueError(f"Unknown solver: {self.solver}")

        self.intercept_ = weights[0].item()
        self.coefs_ = weights[1:].flatten()

    def _gradient_descent(self, X, y, w):
        for i in range(self.iters):
            z = X @ w
            h = self._sigmoid(z)
            gradient = (X.T @ (h - y)) / len(y)

            gradient += self._get_reg_term(w)

            new_w = w - self.lr * gradient
            if np.linalg.norm(new_w - w) < self.tol:
                break
            w = new_w
        return w

    def _stochastic_gradient_descent(self, X, y, w):
        n = len(y)
        for epoch in range(self.iters):
            indices = np.random.permutation(n)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            for i in range(n):
                xi = X_shuffled[i:i+1]
                yi = y_shuffled[i:i+1]

                h = self._sigmoid(xi @ w)
                gradient = xi.T @ (h - yi)

                gradient += self._get_reg_term(w)

                new_w = w - self.lr * gradient
                if np.linalg.norm(new_w - w) < self.tol / n:
                    break
                w = new_w
            else:
                continue
            break
        return w

    def _newton_method(self, X, y, w):
        if self.l1_ratio > 0:
            warnings.warn("Newton's method is not directly suited for L1 regularization. "
                          "Only the L2 part of Elastic Net will be included in the Hessian matrix.")

        for i in range(self.iters):
            z = X @ w
            h = self._sigmoid(z)


            gradient = (X.T @ (h - y)) / len(y)
            gradient += self._get_reg_term(w)


            diag_vec = (h * (1 - h)).flatten()
            H = (X.T * diag_vec) @ X / len(y)

            reg_matrix = np.eye(w.shape[0]) * (1 - self.l1_ratio) * self.alpha
            reg_matrix[0, 0] = 0
            H += reg_matrix

            try:
                step = np.linalg.solve(H, gradient)
                new_w = w - step
            except np.linalg.LinAlgError:
                warnings.warn("Hessian matrix is singular, stopping Newton's method.")
                break # Если матрица вырождена

            if np.linalg.norm(new_w - w) < self.tol:
                break
            w = new_w
        return w

    def predict_proba(self, X: np.array):
        if X.ndim == 1:
            X = X.reshape(1, -1)

        X_inc = np.hstack([np.ones((X.shape[0], 1)), X])
        weights_full = np.concatenate([np.array([self.intercept_]), self.coefs_]).reshape(-1, 1)
        return self._sigmoid(X_inc @ weights_full)

    def predict(self, X: np.array):
        return (self.predict_proba(X) >= 0.5).astype(int).flatten()

    def score(self, X: np.array, y: np.array):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

**1. Градиентный спуск (GD):**

  •  Применимость: Универсален.
  
  •  L2: Работает идеально, так как градиент 2w определен везде.

  •  L1: Функция |w| не дифференцируема в нуле. Мы используем субградиент (np.sign(w)). В точке 0 производная не определена, поэтому GD может "колебаться" около нуля, но для практических задач этого достаточно.

**2. Стохастический градиентный спуск (SGD):**

  •  Применимость: Лучший выбор для огромных датасетов.

  •  L1/L2: Применим аналогично GD. Из-за шума обновлений на каждом объекте SGD еще сложнее сойтись в точный ноль при L1, поэтому веса могут быть "почти нулевыми", но не зануляться идеально без специальных проксимальных операторов.

**3. Метод Ньютона (Newton):**

  •  Применимость: Очень быстрая сходимость (за 5-10 итераций) на гладких функциях.

  •  L2: Идеально. Вторая производная L2-штрафа — это константа (α), которая просто добавляется к диагонали Гессиана (Ridge-регуляризация), что делает матрицу более устойчивой (обусловленной).

  •  L1: Не применим в классическом виде. Вторая производная |w| равна 0 везде, кроме точки 0, где она не существует. Метод Ньютона полагается на аппроксимацию функции параболой, а функция с L1-штрафом имеет "излом". Для L1 обычно используют Coordinate Descent или Proximal Newton Methods.


Продемонстрировать применение реализованного класса на датасете про пингвинов (целевая переменная — вид пингвина). Рассмотреть все возможные варианты (регуляризация/оптимизация). Для категориального признака `island` реализовать самостоятельно преобразование `Target Encoder`, сравнить результаты классификации с `one-hot`. В качестве метрики использовать `f1-score`.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
df = pd.read_csv('/content/drive/MyDrive/магистратура/1 сем/матстат/лабы/lab5/penguins_binary_classification.csv')

In [57]:
df.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'year'],
      dtype='object')

In [58]:
try:
    df = pd.read_csv('penguins.csv')
except FileNotFoundError:
    print("penguins.csv not found, using a sample DataFrame structure for demonstration.")
    data = {
        'species': np.random.choice(['Adelie', 'Gentoo', 'Chinstrap'], 300),
        'island': np.random.choice(['Torgersen', 'Biscoe', 'Dream'], 300),
        'bill_length_mm': np.random.rand(300) * 20 + 30,
        'bill_depth_mm': np.random.rand(300) * 10 + 10,
        'flipper_length_mm': np.random.rand(300) * 50 + 170,
        'body_mass_g': np.random.rand(300) * 2000 + 3000,
        'sex': np.random.choice(['MALE', 'FEMALE', np.nan], 300, p=[0.45, 0.45, 0.1]),
        'year': np.random.choice([2007, 2008, 2009], 300)
    }
    df = pd.DataFrame(data)

df = df.dropna()

original_len = len(df)
df = df[df['species'].isin(['Adelie', 'Gentoo'])].copy()
if len(df) == 0:
    raise ValueError("No 'Adelie' or 'Gentoo' species found after filtering. Please check your dataset.")

df['target'] = (df['species'] == 'Gentoo').astype(int)

if df['year'].nunique() == 1:
    X_cols = [col for col in df.columns if col not in ['species', 'target', 'year']]
else:
    X_cols = [col for col in df.columns if col not in ['species', 'target']]

X = df[X_cols]
y = df['target']

penguins.csv not found, using a sample DataFrame structure for demonstration.


In [59]:
class MyTargetEncoder:
    def __init__(self):
        self.mapping = {}
        self.global_mean = 0

    def fit(self, column, target):
        self.global_mean = target.mean()
        self.mapping = target.groupby(column).mean().to_dict()

    def transform(self, column):
        return column.map(self.mapping).fillna(self.global_mean)


def prepare_data(df_input, y_target, encoding_type='ohe'):
    df_proc = df_input.copy()

    if 'sex' in df_proc.columns:
        df_proc['sex'] = (df_proc['sex'] == 'MALE').astype(int)

    if 'island' in df_proc.columns:
        if encoding_type == 'target':
            te = MyTargetEncoder()
            te.fit(df_proc['island'], y_target)
            df_proc['island'] = te.transform(df_proc['island'])
        else:
            df_proc = pd.get_dummies(df_proc, columns=['island'], drop_first=True, dtype=int)

    return df_proc

In [60]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_ohe = prepare_data(X_train_raw, y_train, encoding_type='ohe')
X_test_ohe = prepare_data(X_test_raw, y_test, encoding_type='ohe')

X_train_te = prepare_data(X_train_raw, y_train, encoding_type='target')
X_test_te = prepare_data(X_test_raw, y_test, encoding_type='target')

scaler_ohe = StandardScaler()
X_train_ohe_scaled = pd.DataFrame(scaler_ohe.fit_transform(X_train_ohe), columns=X_train_ohe.columns, index=X_train_ohe.index)
X_test_ohe_scaled = pd.DataFrame(scaler_ohe.transform(X_test_ohe), columns=X_test_ohe.columns, index=X_test_ohe.index)

scaler_te = StandardScaler()
X_train_te_scaled = pd.DataFrame(scaler_te.fit_transform(X_train_te), columns=X_train_te.columns, index=X_train_te.index)
X_test_te_scaled = pd.DataFrame(scaler_te.transform(X_test_te), columns=X_test_te.columns, index=X_test_te.index)

In [61]:
solvers = ['gd', 'sgd', 'newton']
reg_types = {
    'l2': 0.0,
    'l1': 1.0,
    'l1l2': 0.5
}

results = []

for enc_name, X_train_scaled, X_test_scaled in [('OHE', X_train_ohe_scaled, X_test_ohe_scaled),
                                                ('Target', X_train_te_scaled, X_test_te_scaled)]:
    for s in solvers:
        for r_name, l1_r in reg_types.items():

            print(f"Running: Encoding={enc_name}, Solver={s}, Reg={r_name}")

            model = MyBinaryLogisticRegression(
                solver=s,
                l1_ratio=l1_r,
                alpha=0.1,
                iters=1000,
                lr=0.01
            )

            try:
                model.fit(X_train_scaled, y_train)
                preds = model.predict(X_test_scaled.values)
                f1 = f1_score(y_test, preds)

                results.append({
                    'Encoding': enc_name,
                    'Solver': s,
                    'Reg': r_name,
                    'F1-Score': round(f1, 4)
                })
            except Exception as e:
                print(f"Error with Encoding={enc_name}, Solver={s}, Reg={r_name}: {e}")


res_df = pd.DataFrame(results)
print("\n--- Результаты классификации ---")
print(res_df.sort_values(by='F1-Score', ascending=False))

Running: Encoding=OHE, Solver=gd, Reg=l2
Running: Encoding=OHE, Solver=gd, Reg=l1
Running: Encoding=OHE, Solver=gd, Reg=l1l2
Running: Encoding=OHE, Solver=sgd, Reg=l2
Running: Encoding=OHE, Solver=sgd, Reg=l1
Running: Encoding=OHE, Solver=sgd, Reg=l1l2
Running: Encoding=OHE, Solver=newton, Reg=l2
Running: Encoding=OHE, Solver=newton, Reg=l1
Running: Encoding=OHE, Solver=newton, Reg=l1l2
Running: Encoding=Target, Solver=gd, Reg=l2
Running: Encoding=Target, Solver=gd, Reg=l1
Running: Encoding=Target, Solver=gd, Reg=l1l2
Running: Encoding=Target, Solver=sgd, Reg=l2


/tmp/ipython-input-1909001358.py:92: UserWarning: Newton's method is not directly suited for L1 regularization. Only the L2 part of Elastic Net will be included in the Hessian matrix.
  warnings.warn("Newton's method is not directly suited for L1 regularization. "
/tmp/ipython-input-1909001358.py:92: UserWarning: Newton's method is not directly suited for L1 regularization. Only the L2 part of Elastic Net will be included in the Hessian matrix.
  warnings.warn("Newton's method is not directly suited for L1 regularization. "
/tmp/ipython-input-1909001358.py:115: UserWarning: Hessian matrix is singular, stopping Newton's method.
  warnings.warn("Hessian matrix is singular, stopping Newton's method.")


Running: Encoding=Target, Solver=sgd, Reg=l1
Running: Encoding=Target, Solver=sgd, Reg=l1l2
Running: Encoding=Target, Solver=newton, Reg=l2
Running: Encoding=Target, Solver=newton, Reg=l1
Running: Encoding=Target, Solver=newton, Reg=l1l2

--- Результаты классификации ---
   Encoding  Solver   Reg  F1-Score
14   Target     sgd  l1l2    0.7083
9    Target      gd    l2    0.6809
12   Target     sgd    l2    0.6809
15   Target  newton    l2    0.6809
13   Target     sgd    l1    0.6557
1       OHE      gd    l1    0.6557
10   Target      gd    l1    0.6557
2       OHE      gd  l1l2    0.6557
8       OHE  newton  l1l2    0.6557
4       OHE     sgd    l1    0.6557
11   Target      gd  l1l2    0.6557
7       OHE  newton    l1    0.6364
16   Target  newton    l1    0.6222
5       OHE     sgd  l1l2    0.5926
3       OHE     sgd    l2    0.5778
0       OHE      gd    l2    0.5652
6       OHE  newton    l2    0.5652
17   Target  newton  l1l2    0.2632


/tmp/ipython-input-1909001358.py:92: UserWarning: Newton's method is not directly suited for L1 regularization. Only the L2 part of Elastic Net will be included in the Hessian matrix.
  warnings.warn("Newton's method is not directly suited for L1 regularization. "
/tmp/ipython-input-1909001358.py:115: UserWarning: Hessian matrix is singular, stopping Newton's method.
  warnings.warn("Hessian matrix is singular, stopping Newton's method.")
/tmp/ipython-input-1909001358.py:92: UserWarning: Newton's method is not directly suited for L1 regularization. Only the L2 part of Elastic Net will be included in the Hessian matrix.
  warnings.warn("Newton's method is not directly suited for L1 regularization. "


# Теоретическая часть

Пусть данные имеют вид
$$
(x_i, y_i), \quad y_i \in \{1, \ldots,M\}, \quad i \in \{1, \ldots, N\},
$$
причем первая координата набора признаков каждого объекта равна $1$.
Используя `softmax`-подход, дискриминативная модель имеет следующий вид
$$
\mathbb P(C_k|x) = \frac{\exp(\omega_k^Tx)}{\sum_i \exp(\omega_i^Tx)}.
$$
Для написания правдоподобия удобно провести `one-hot` кодирование меток класса, сопоставив каждому объекту $x_i$ вектор $\widehat y_i = (y_{11}, \ldots, y_{1M})$ длины $M$, состоящий из нулей и ровно одной единицы ($y_{iy_i} = 1$), отвечающей соответствующему классу. В этом случае правдоподобие имеет вид
$$
\mathbb P(D|\omega) = \prod_{i = 1}^{N}\prod_{j = 1}^M \mathbb P(C_j|x_i)^{y_{ij}}.
$$
Ваша задача: вывести функцию потерь, градиент и гессиан для многоклассовой логистической регрессии. Реализовать матрично. На синтетическом примере продемонстрировать работу алгоритма, построить гиперплоскости, объяснить классификацию